<a href="https://colab.research.google.com/github/raman-deep-kaur/Monthly-Rebalance-Strategy/blob/main/221230054_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
@author: A2NG Services
"""

# DO NOT add or remove any libraries
import pandas as pd
import numpy as np
import datetime

# Do not add any function outside the class "TradingStrategy"
class TradingStrategy:
    #====================== DO NOT Change this code =========================#
    def __init__(self,rebalancing_frequency="M",initial_capital=10000000,start_date=datetime.datetime(2010,1,4),end_date=datetime.datetime(2023,12,29)):
        self.rebalancing_frequency=rebalancing_frequency
        self.initial_capital=initial_capital
        self.start_date=start_date
        self.end_date=end_date

        self.load_pricing_data()

        self.trades=pd.DataFrame(columns=['Ticker','Date','Price','n_shares_traded','Signal'])
        self.n_shares_position=pd.DataFrame(columns=self.prices.columns,index=self.prices.index)
        self.position_amount=pd.DataFrame(columns=list(self.prices.columns)+["Cash"],index=self.prices.index)
        self.position_atp=pd.DataFrame(columns=self.prices.columns,index=self.prices.index)
        self.position_long_short=pd.DataFrame(columns=self.prices.columns,index=self.prices.index)


    def load_pricing_data(self):
        # You can change the parameters of read_csv function if needed
        self.prices=pd.read_csv("Prices.csv",index_col='Date',parse_dates=True,dayfirst=False)

    def get_trades(self):
        return self.trades

    def get_positions(self):
        return self.n_shares_position

    def get_position_amount(self):
        return self.position_amount

    def get_cash(self):
        return self.position_amount['Cash']

    #=====================================================#
    #=============Write your code from here===============#


    def run_backtest(self): # DO NOT change the name of this function

        prices = self.prices.copy()[(self.prices.index >= self.start_date) & (self.prices.index <= self.end_date)]

        # Fill missing values
        prices = prices.fillna(method='ffill').dropna(how='all', axis=1)

        # Calculate 3-month momentum (~63 trading days)
        momentum = prices.pct_change(periods=63)

        # Generate rebalancing dates based on frequency
        rebalancing_dates = prices.groupby([prices.index.year, prices.index.month]).apply(lambda x: x.index[0])
        rebalancing_dates = pd.to_datetime(rebalancing_dates.values)

        print("✅ Number of rebalancing periods:", len(rebalancing_dates))
        print("📅 Sample rebalancing dates:", rebalancing_dates[:5])
        print("📊 Momentum matrix shape:", momentum.shape)
        print("📈 Example momentum values on first rebalance date:\n", momentum.loc[rebalancing_dates[0]].dropna().sort_values(ascending=False).head())

        for date in rebalancing_dates:
            if date not in momentum.index:
                continue  # Skip if not enough lookback data

            daily_momentum = momentum.loc[date].dropna()

            if len(daily_momentum) < 20:
                continue  # Skip if not enough stocks

            long_stocks = daily_momentum.nlargest(10).index.tolist()
            short_stocks = daily_momentum.nsmallest(10).index.tolist()

            print(f"\n📅 Rebalancing on {date.date()}")
            print("📈 Long Stocks:", long_stocks)
            print("📉 Short Stocks:", short_stocks)

            capital = self.initial_capital
            weight = 0.05  # 5% per stock
            cash_remaining = capital

            for ticker, signal in zip(long_stocks + short_stocks, [1]*10 + [-1]*10):
                price = prices.loc[date, ticker]
                capital_allocated = capital * weight
                n_shares = int(capital_allocated // price)

                if n_shares == 0:
                    continue  # Skip if price too high for any shares

                trade_value = n_shares * price
                cash_remaining -= trade_value

                # Record the trade if valid
                new_trade = pd.DataFrame([{
                    'Ticker': ticker,
                    'Date': date,
                    'Price': price,
                    'n_shares_traded': n_shares,
                    'Signal': signal
                }])
                if not new_trade.empty:
                    self.trades = pd.concat([self.trades, new_trade], ignore_index=True)

                self.n_shares_position.loc[date, ticker] = signal * n_shares
                self.position_amount.loc[date, ticker] = signal * n_shares * price
                self.position_atp.loc[date, ticker] = price
                self.position_long_short.loc[date, ticker] = signal

            self.position_amount.loc[date, "Cash"] = cash_remaining
            print(f"💰 Cash after trades on {date.date()}: {cash_remaining:,.2f}")

        print("📃 Latest trades:\n", self.trades.tail(5))


if __name__=="__main__":
    ts=TradingStrategy(rebalancing_frequency="M",initial_capital=10000000,start_date=datetime.datetime(2010,1,4),end_date=datetime.datetime(2023,12,29))
    ts.run_backtest()


<ipython-input-26-d227f70fad4a>:54: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  prices = prices.fillna(method='ffill').dropna(how='all', axis=1)
<ipython-input-26-d227f70fad4a>:108: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.trades = pd.concat([self.trades, new_trade], ignore_index=True)


✅ Number of rebalancing periods: 168
📅 Sample rebalancing dates: DatetimeIndex(['2010-01-04', '2010-02-01', '2010-03-02', '2010-04-01',
               '2010-05-03'],
              dtype='datetime64[ns]', freq=None)
📊 Momentum matrix shape: (3474, 78)
📈 Example momentum values on first rebalance date:
 Series([], Name: 2010-01-04 00:00:00, dtype: float64)

📅 Rebalancing on 2010-05-03
📈 Long Stocks: ['S25', 'S21', 'S71', 'S27', 'S64', 'S10', 'S41', 'S2', 'S13', 'S4']
📉 Short Stocks: ['S70', 'S36', 'S14', 'S49', 'S46', 'S23', 'S16', 'S1', 'S51', 'S50']
💰 Cash after trades on 2010-05-03: 1,252.99

📅 Rebalancing on 2010-06-01
📈 Long Stocks: ['S33', 'S25', 'S21', 'S10', 'S27', 'S64', 'S29', 'S11', 'S72', 'S71']
📉 Short Stocks: ['S70', 'S30', 'S69', 'S14', 'S49', 'S35', 'S74', 'S75', 'S76', 'S23']
💰 Cash after trades on 2010-06-01: 1,491.94

📅 Rebalancing on 2010-07-01
📈 Long Stocks: ['S33', 'S25', 'S21', 'S64', 'S10', 'S27', 'S40', 'S29', 'S24', 'S15']
📉 Short Stocks: ['S69', 'S76', 'S74', '

Saving Prices.csv to Prices.csv


In [ ]:
def load_pricing_data(self):
    self.prices = pd.read_csv("Prices.csv", index_col='Date', parse_dates=True, dayfirst=False)


In [ ]:
from google.colab import files

# After your backtest code runs and files are saved, download them like this:
files.download("trades.csv")

In [ ]:
# Add this code after the line `ts.run_backtest()`

# Save the trades DataFrame to a CSV file
ts.get_trades().to_csv('trades.csv', index=False)

# Download the saved trades CSV file
from google.colab import files
files.download('trades.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>